In [1]:
import sqlite3 as db
import pandas as pd
conn = db.connect('practice.db')
c = conn.cursor()
%sql sqlite:///practice.db

UsageError: Line magic function `%sql` not found.


In [2]:
c.execute('DROP TABLE IF EXISTS EmployeeDemographics')
c.execute(
"""
Create Table EmployeeDemographics 
(EmployeeID int, 
FirstName varchar(50), 
LastName varchar(50), 
Age int, 
Gender varchar(50)
)
""" 
)

In [3]:
c.execute('DROP TABLE IF EXISTS EmployeeSalary')
c.execute(
"""
Create Table EmployeeSalary 
(EmployeeID int, 
JobTitle varchar(50), 
Salary int
)
""" 
)

In [5]:
# Create a new table to track department data. Adjust schema as needed:
c.execute('DROP TABLE IF EXISTS EmployeeDepartment')
c.execute("""
CREATE TABLE IF NOT EXISTS EmployeeDepartment (
    emp_id INTEGER PRIMARY KEY,
    department TEXT,
    hire_date TEXT,
    manager_id INTEGER
);
""")

In [ ]:
c.execute("""
Insert into EmployeeDemographics VALUES
(1001, 'Jim', 'Halpert', 30, 'Male'),
(1002, 'Pam', 'Beasley', 30, 'Female'),
(1003, 'Dwight', 'Schrute', 29, 'Male'),
(1004, 'Angela', 'Martin', 31, 'Female'),
(1005, 'Toby', 'Flenderson', 32, 'Male'),
(1006, 'Michael', 'Scott', 35, 'Male'),
(1007, 'Meredith', 'Palmer', 32, 'Female'),
(1008, 'Stanley', 'Hudson', 38, 'Male'),
(1009, 'Kevin', 'Malone', 31, 'Male'),
(1010, 'John', 'Smith', 35, 'Male'),
(1011, 'Jane', 'Johnson', 28, 'Female'),
(1012, 'Michael', 'Williams', 40, 'Male'),
(1013, 'Emily', 'Brown', 26, 'Female'),
(1014, 'David', 'Jones', 33, 'Male'),
(1015, 'Sarah', 'Garcia', 31, 'Female'),
(1016, 'James', 'Miller', 45, 'Male'),
(1017, 'Anna', 'Davis', 29, 'Female'),
(1018, 'Robert', 'Rodriguez', 38, 'Male'),
(1019, 'Laura', 'Martinez', 30, 'Female'),
(1020, 'Oscar', 'Martinez', 37, 'Male'),
(1021, 'Phyllis', 'Vance', 45, 'Female'),
(1022, 'Kelly', 'Kapoor', 28, 'Female'),
(1023, 'Ryan', 'Howard', 28, 'Male'),
(1024, 'Erin', 'Hannon', 25, 'Female');
""")
c.execute(
"""
Insert Into EmployeeSalary VALUES
(1001, 'Salesman', 45000),
(1002, 'Receptionist', 36000),
(1003, 'Salesman', 63000),
(1004, 'Accountant', 47000),
(1005, 'HR', 50000),
(1006, 'Regional Manager', 65000),
(1007, 'Supplier Relations', 41000),
(1008, 'Salesman', 48000),
(1009, 'Accountant', 42000),
(1010, 'Data Analyst', 95409),
(1011, 'Sales Representative', 113332),
(1012, 'Sales Representative', 76199),
(1013, 'Sales Representative', 100110),
(1014, 'Operations Manager', 98577),
(1015, 'HR', 72000),
(1016, 'Software Engineer', 110500),
(1017, 'IT Support', 65000),
(1018, 'Marketing Manager', 87000),
(1019, 'Finance Analyst', 93000),
(1020, 'Accountant', 52000),
(1021, 'Sales Representative', 48000),
(1022, 'Customer Service Rep', 42000),
(1023, 'Temp', 35000),
(1024, 'Receptionist', 34000);
""" 
)


# Insert existing 19 employees (IDs 1001–1019) into EmployeeDepartment:
c.execute("""
INSERT INTO EmployeeDepartment VALUES
(1001, 'Sales', '2019-03-15', 1006),
(1002, 'Reception', '2019-04-20', 1006),
(1003, 'Sales', '2019-05-10', 1006),
(1004, 'Accounting', '2020-01-05', 1004),
(1005, 'HR', '2018-11-12', 1015),
(1006, 'Management', '2010-04-15', 1006),
(1007, 'Supplier Relations', '2019-07-07', 1007),
(1008, 'Sales', '2020-06-22', 1006),
(1009, 'Accounting', '2021-03-19', 1004),
(1010, 'Data', '2022-01-01', 1014),
(1011, 'Sales', '2022-02-15', 1014),
(1012, 'Sales', '2022-04-01', 1014),
(1013, 'Sales', '2022-04-01', 1014),
(1014, 'Operations', '2021-12-01', 1014),
(1015, 'HR', '2018-11-12', 1015),
(1016, 'IT', '2023-02-10', 1014),
(1017, 'IT', '2023-02-11', 1016),
(1018, 'Marketing', '2021-05-27', 1014),
(1019, 'Finance', '2022-07-15', 1014);
""")

In [8]:
conn.commit()

## Extra Practice

**Joins and SubQueries**

Question 1: 

Find the employees who earn more than the average salary of their respective department.

(Hint: Use a subquery to calculate the average salary for each department.)

In [20]:
# we have a tie
# ties can be broken by using a series of ORDER BY state,emts
query = """
WITH dept_avg AS (SELECT JobTitle, AVG(Salary) as dept_salary
FROM EmployeeSalary
GROUP BY JobTitle)

SELECT dem.EmployeeID, dem.FirstName, sal.Salary, sal.JobTitle, dept_avg.dept_salary
FROM EmployeeSalary AS sal
JOIN EmployeeDemographics AS dem 
    ON sal.EmployeeID = dem.EmployeeID
JOIN dept_avg
    ON sal.JobTitle = dept_avg.JobTitle
WHERE sal.Salary > dept_avg.dept_salary
ORDER BY sal.JobTitle, sal.Salary DESC;
""" 

df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,FirstName,Salary,JobTitle,dept_salary
0,1020,Oscar,52000,Accountant,47000.00
1,1015,Sarah,72000,HR,61000.00
2,1002,Pam,36000,Receptionist,35000.00
3,1011,Jane,113332,Sales Representative,84410.25
4,1013,Emily,100110,Sales Representative,84410.25
5,1003,Dwight,63000,Salesman,52000.00


Question 2:

List employees who work under a manager who earns less than them.

(Hint: Join EmployeeDemographics, EmployeeSalary, and EmployeeDepartment to compare salaries with managers.)

In [47]:
query = """

WITH managerSal AS (
    SELECT dept.manager_id, sal.Salary
    FROM EmployeeDepartment as dept
    JOIN EmployeeSalary as sal
        ON dept.emp_id = sal.EmployeeID
    JOIN EmployeeDemographics as dem
        ON dem.EmployeeID = dept.emp_id
    GROUP BY dept.manager_id
)
SELECT dem.EmployeeID, dem.FirstName, sal.Salary, dept.manager_id, managerSal.Salary
FROM EmployeeDepartment as dept
JOIN EmployeeSalary as sal
    ON dept.emp_id = sal.EmployeeID
JOIN EmployeeDemographics as dem
    ON dem.EmployeeID = dept.emp_id
JOIN managerSal
    ON dept.manager_id = managerSal.manager_id
WHERE sal.Salary > managerSal.Salary
ORDER BY sal.Salary
"""

df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,FirstName,Salary,manager_id,Salary
0,1008,Stanley,48000,1006,45000
1,1003,Dwight,63000,1006,45000
2,1006,Michael,65000,1006,45000
3,1015,Sarah,72000,1015,50000
4,1014,David,98577,1014,95409
5,1013,Emily,100110,1014,95409
6,1016,James,110500,1014,95409
7,1011,Jane,113332,1014,95409


# Course Work

In [163]:
query = """
SELECT *
FROM EmployeeDemographics;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,FirstName,LastName,Age,Gender
0,1001,Jim,Halpert,30,Male
1,1002,Pam,Beasley,30,Female
2,1003,Dwight,Schrute,29,Male
3,1004,Angela,Martin,31,Female
4,1005,Toby,Flenderson,32,Male
5,1006,Michael,Scott,35,Male
6,1007,Meredith,Palmer,32,Female
7,1008,Stanley,Hudson,38,Male
8,1009,Kevin,Malone,31,Male
9,1010,John,Smith,35,Male


In [164]:
query = """
SELECT *
FROM EmployeeSalary;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,JobTitle,Salary
0,1001,Salesman,45000
1,1002,Receptionist,36000
2,1003,Salesman,63000
3,1004,Accountant,47000
4,1005,HR,50000
5,1006,Regional Manager,65000
6,1007,Supplier Relations,41000
7,1008,Salesman,48000
8,1009,Accountant,42000
9,1010,Data Analyst,95409


#### Group By

Group By Clause groups together rows with the same values so we can later apply aggregations.

In [30]:
query = """
SELECT Gender, AVG(Age) AS avg_age
FROM EmployeeDemographics
GROUP BY Gender;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,Gender,avg_age
0,Female,29.625000
1,Male,35.090909


The below groups similar rows together. Seeing as how there could be multiple JobTitle's with the same salary these get consolidated.

And then we can perform aggregations.

In [40]:
query = """
SELECT JobTitle, Salary
FROM EmployeeSalary
GROUP BY JobTitle, Salary;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,JobTitle,Salary
0,Accountant,42000
1,Accountant,47000
2,Data Analyst,95409
3,Finance Analyst,93000
4,HR,50000
5,HR,72000
6,IT Support,65000
7,Marketing Manager,87000
8,Operations Manager,98577
9,Receptionist,36000


The below breaks down a series of aggregations and counts around the Gender category.

In [45]:
query = """
SELECT Gender, AVG(Age) as avg_age, MAX(Age) as max_age, MIN(Age) as min_age, COUNT(Age) as count
FROM EmployeeDemographics
GROUP BY Gender
""" 
df = pd.read_sql_query(query, conn)
display(df)

,Gender,avg_age,max_age,min_age,count
0,Female,29.625000,32,26,8
1,Male,35.090909,45,29,11


**Order of Groupings**

The following get ordered according to their occurence within the ORDER BY clause

In [55]:
query = """
SELECT *
FROM EmployeeDemographics
ORDER BY Gender, Age DESC
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,FirstName,LastName,Age,Gender
0,1007,Meredith,Palmer,32,Female
1,1004,Angela,Martin,31,Female
2,1015,Sarah,Garcia,31,Female
3,1002,Pam,Beasley,30,Female
4,1019,Laura,Martinez,30,Female
5,1017,Anna,Davis,29,Female
6,1011,Jane,Johnson,28,Female
7,1013,Emily,Brown,26,Female
8,1016,James,Miller,45,Male
9,1012,Michael,Williams,40,Male


#### HAVING vs WHERE

In the context of the WHERE clause aggregations aren't applied until the GROUP BY clause is executed.

This is where HAVING comes into play.

The HAVING clause is used after the GROUP BY claus eand can be used to filter on aggregated functions.

In [60]:
query = """
SELECT Gender, AVG(Age)
FROM EmployeeDemographics
GROUP BY Gender
HAVING AVG(Age) > 30
""" 
df = pd.read_sql_query(query, conn)
display(df)

,Gender,AVG(Age)
0,Male,35.090909


The below Query first groups the data by JobTitle thus removing non unique rows.

Next the query performs two filtering operations:

    1.) It filters for JobTitle's containing the word Manager
    2.) Filters for avg salary greater than 70,000

In [67]:
query = """
SELECT JobTitle, AVG(Salary) as avg_salary
FROM EmployeeSalary
WHERE JobTitle LIKE '%Manager%'
GROUP BY JobTitle
HAVING AVG(Salary) > 70000
""" 
df = pd.read_sql_query(query, conn)
display(df)

,JobTitle,avg_salary
0,Marketing Manager,87000.0
1,Operations Manager,98577.0


#### Joining Tables

The default join type is an **inner join** as seen below.

Remember: inner join only columns where the rows that have the same values in both columns

In [79]:
query = """
SELECT *
FROM EmployeeSalary AS sal
INNER JOIN EmployeeDemographics AS dem
    ON sal.EmployeeID = dem.EmployeeID  
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,JobTitle,Salary,EmployeeID,FirstName,LastName,Age,Gender
0,1001,Salesman,45000,1001,Jim,Halpert,30,Male
1,1002,Receptionist,36000,1002,Pam,Beasley,30,Female
2,1003,Salesman,63000,1003,Dwight,Schrute,29,Male
3,1004,Accountant,47000,1004,Angela,Martin,31,Female
4,1005,HR,50000,1005,Toby,Flenderson,32,Male
5,1006,Regional Manager,65000,1006,Michael,Scott,35,Male
6,1007,Supplier Relations,41000,1007,Meredith,Palmer,32,Female
7,1008,Salesman,48000,1008,Stanley,Hudson,38,Male
8,1009,Accountant,42000,1009,Kevin,Malone,31,Male
9,1010,Data Analyst,95409,1010,John,Smith,35,Male


Note: we have to be specific when calling rows otherwise the system will throw an error as it does not know where to pull data from.

In [81]:
query = """
SELECT dem.FirstName, dem.Age, dem.Gender, sal.Salary
FROM EmployeeSalary AS sal
INNER JOIN EmployeeDemographics AS dem
    ON sal.EmployeeID = dem.EmployeeID  
""" 
df = pd.read_sql_query(query, conn)
display(df)

,FirstName,Age,Gender,Salary
0,Jim,30,Male,45000
1,Pam,30,Female,36000
2,Dwight,29,Male,63000
3,Angela,31,Female,47000
4,Toby,32,Male,50000
5,Michael,35,Male,65000
6,Meredith,32,Female,41000
7,Stanley,38,Male,48000
8,Kevin,31,Male,42000
9,John,35,Male,95409


**Outer Joins**

For Outer Joins we have a Left Outer and a Right Outer join.

    - Left Join: takes everything from the left table and will only return matches from the right table
    - Right Join: takes everything from the right table and will only return matches from the left table

In [84]:
query = """
SELECT dem.FirstName, dem.Age, dem.Gender, sal.Salary
FROM EmployeeSalary AS sal
LEFT OUTER JOIN EmployeeDemographics AS dem
    ON sal.EmployeeID = dem.EmployeeID  
""" 
df = pd.read_sql_query(query, conn)
display(df)

,FirstName,Age,Gender,Salary
0,Jim,30,Male,45000
1,Pam,30,Female,36000
2,Dwight,29,Male,63000
3,Angela,31,Female,47000
4,Toby,32,Male,50000
5,Michael,35,Male,65000
6,Meredith,32,Female,41000
7,Stanley,38,Male,48000
8,Kevin,31,Male,42000
9,John,35,Male,95409


#### Unions

Unions allow us to combine rows together. Not like Columns with Joins.

We can combine rows together from separate or the same table.

For UNIONS the data should be the same when combining.

By default this is a UNION DISTINCT: meaning all the duplicates get removed.

In [91]:
query = """
SELECT EmployeeID
FROM EmployeeSalary
UNION
SELECT EmployeeID
FROM EmployeeDemographics;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID
0,1001
1,1002
2,1003
3,1004
4,1005
5,1006
6,1007
7,1008
8,1009
9,1010


#### SubQueries

We are going to use a SubQuery to filter rows of data when the employee's age is equal to 30.

We want to return the employeeID's of employees whose age is equal to 30 from the EmployeeDemographics table.

**SubQuery in a WHERE clause**

In [100]:
query = """
SELECT *
FROM EmployeeSalary
WHERE EmployeeID IN
    (
        SELECT EmployeeID
        FROM EmployeeDemographics
        WHERE Age = 30
    );
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,JobTitle,Salary
0,1001,Salesman,45000
1,1002,Receptionist,36000
2,1019,Finance Analyst,93000


I want to include another JOIN statement before the WHERE clause to pull in First and Last name.

In [107]:
query = """
SELECT dem.EmployeeID, sal.JobTitle, sal.Salary, dem.FirstName, dem.LastName, dem.Age, dem.Gender
FROM EmployeeSalary AS sal
JOIN EmployeeDemographics as dem
    ON sal.EmployeeID = dem.EmployeeID
    WHERE dem.EmployeeID IN
        (
            SELECT EmployeeID
            FROM EmployeeDemographics
            WHERE Age = 30
        );
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,JobTitle,Salary,FirstName,LastName,Age,Gender
0,1001,Salesman,45000,Jim,Halpert,30,Male
1,1002,Receptionist,36000,Pam,Beasley,30,Female
2,1019,Finance Analyst,93000,Laura,Martinez,30,Female


#### Window Functions

These are similar to a Group By - allow us to look at a partition or a row.

We want to take Gender and compare it to the actual salaries

In [116]:
# currently looks at an average salary over everyone (i.e. all rows)
query = """
SELECT dem.Gender, AVG(sal.Salary) OVER()
FROM EmployeeDemographics AS dem
JOIN EmployeeSalary AS sal
    ON dem.EmployeeID = sal.EmployeeID;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,Gender,AVG(sal.Salary) OVER()
0,Male,70954.052632
1,Female,70954.052632
2,Male,70954.052632
3,Female,70954.052632
4,Male,70954.052632
5,Male,70954.052632
6,Female,70954.052632
7,Male,70954.052632
8,Male,70954.052632
9,Male,70954.052632


As a next step we can partition by the gender column.

This will not roll everything up but it will perform the average Salary calculation across every row in our data set.

The benefit is we can add additional information i.e. (more rows) to get granular insight into our dataset without impacting the PARTITION.

In [124]:
query = """
SELECT dem.FirstName, dem.LastName, dem.Gender, AVG(sal.Salary) OVER(PARTITION BY dem.Gender)
FROM EmployeeDemographics AS dem
JOIN EmployeeSalary AS sal
    ON dem.EmployeeID = sal.EmployeeID;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,FirstName,LastName,Gender,AVG(sal.Salary) OVER(PARTITION BY dem.Gender)
0,Pam,Beasley,Female,70930.250000
1,Angela,Martin,Female,70930.250000
2,Meredith,Palmer,Female,70930.250000
3,Jane,Johnson,Female,70930.250000
4,Emily,Brown,Female,70930.250000
5,Sarah,Garcia,Female,70930.250000
6,Anna,Davis,Female,70930.250000
7,Laura,Martinez,Female,70930.250000
8,Jim,Halpert,Male,70971.363636
9,Dwight,Schrute,Male,70971.363636


Now we are looking at the SUM of total salaries by Gender.

In [127]:
query = """
SELECT dem.FirstName, dem.LastName, dem.Gender, SUM(sal.Salary) OVER(PARTITION BY dem.Gender)
FROM EmployeeDemographics AS dem
JOIN EmployeeSalary AS sal
    ON dem.EmployeeID = sal.EmployeeID;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,FirstName,LastName,Gender,SUM(sal.Salary) OVER(PARTITION BY dem.Gender)
0,Pam,Beasley,Female,567442
1,Angela,Martin,Female,567442
2,Meredith,Palmer,Female,567442
3,Jane,Johnson,Female,567442
4,Emily,Brown,Female,567442
5,Sarah,Garcia,Female,567442
6,Anna,Davis,Female,567442
7,Laura,Martinez,Female,567442
8,Jim,Halpert,Male,780685
9,Dwight,Schrute,Male,780685


**Rolling Total Implementation**

In [131]:
query = """
SELECT dem.FirstName, dem.LastName, dem.Gender, sal.Salary, SUM(sal.Salary) OVER(PARTITION BY dem.Gender ORDER BY sal.EmployeeID) as rolling_total
FROM EmployeeDemographics AS dem
JOIN EmployeeSalary AS sal
    ON dem.EmployeeID = sal.EmployeeID;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,FirstName,LastName,Gender,Salary,rolling_total
0,Pam,Beasley,Female,36000,36000
1,Angela,Martin,Female,47000,83000
2,Meredith,Palmer,Female,41000,124000
3,Jane,Johnson,Female,113332,237332
4,Emily,Brown,Female,100110,337442
5,Sarah,Garcia,Female,72000,409442
6,Anna,Davis,Female,65000,474442
7,Laura,Martinez,Female,93000,567442
8,Jim,Halpert,Male,45000,45000
9,Dwight,Schrute,Male,63000,108000


**ROW_NUMBER()**

Special things you can do with Window Functions

In [134]:
query = """
SELECT dem.EmployeeID, dem.FirstName, dem.LastName, dem.Gender, sal.Salary, ROW_NUMBER() OVER(PARTITION BY dem.Gender)
FROM EmployeeDemographics AS dem
JOIN EmployeeSalary AS sal
    ON dem.EmployeeID = sal.EmployeeID;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,FirstName,LastName,Gender,Salary,ROW_NUMBER() OVER(PARTITION BY dem.Gender)
0,1002,Pam,Beasley,Female,36000,1
1,1004,Angela,Martin,Female,47000,2
2,1007,Meredith,Palmer,Female,41000,3
3,1011,Jane,Johnson,Female,113332,4
4,1013,Emily,Brown,Female,100110,5
5,1015,Sarah,Garcia,Female,72000,6
6,1017,Anna,Davis,Female,65000,7
7,1019,Laura,Martinez,Female,93000,8
8,1001,Jim,Halpert,Male,45000,1
9,1003,Dwight,Schrute,Male,63000,2


**ROW()**

Let's say now we want to rank the rows by the highest salary down to the lowest salary.

Row Number gives a row number based on whatever you're partitioning by.

In [136]:
query = """
SELECT dem.EmployeeID, dem.FirstName, dem.LastName, dem.Gender, sal.Salary, ROW_NUMBER() OVER(PARTITION BY dem.Gender ORDER BY sal.Salary DESC)
FROM EmployeeDemographics AS dem
JOIN EmployeeSalary AS sal
    ON dem.EmployeeID = sal.EmployeeID;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,FirstName,LastName,Gender,Salary,ROW_NUMBER() OVER(PARTITION BY dem.Gender ORDER BY sal.Salary DESC)
0,1011,Jane,Johnson,Female,113332,1
1,1013,Emily,Brown,Female,100110,2
2,1019,Laura,Martinez,Female,93000,3
3,1015,Sarah,Garcia,Female,72000,4
4,1017,Anna,Davis,Female,65000,5
5,1004,Angela,Martin,Female,47000,6
6,1007,Meredith,Palmer,Female,41000,7
7,1002,Pam,Beasley,Female,36000,8
8,1016,James,Miller,Male,110500,1
9,1014,David,Jones,Male,98577,2


**RANK()**

This will give it more of an official ranking.

Resource: https://www.geeksforgeeks.org/rank-function-in-sql-server/

When rank encounters a duplicate value it assigns it the same ranking.

However, for the next proceeding value it assigns it the next number in the position.

**Practice for HW**

In [ ]:
WITH MaxSalary AS (

    SELECT jt1.job_category AS job_category,
         MAX(s1.salary_range_to) AS max_salary
    FROM job_posting jp1
    JOIN job_title jt1
       ON jp1.title_id = jt1.id
    JOIN salary s1
       ON jp1.id = s1.job_posting_id
    GROUP BY jt1.job_category
)

SELECT DISTINCT jt2.business_title, MaxSalary.job_category, s2.salary_range_to, MaxSalary.max_salary
FROM job_posting AS jp
JOIN job_title AS jt2
	ON jp.title_id = jt2.id
JOIN salary AS s2
	ON s2.job_posting_id = jp.id
JOIN MaxSalary
    ON MaxSalary.job_category = jt2.job_category 

In [38]:
query = """
SELECT sal.JobTitle, dem.Gender, sal.Salary,
RANK() OVER(PARTITION BY sal.JobTitle ORDER BY sal.Salary DESC, dem.Age, dem.Gender) as rank
FROM EmployeeDemographics as dem
JOIN EmployeeSalary as sal
    ON dem.EmployeeID = sal.EmployeeID
""" 
df = pd.read_sql_query(query, conn)
display(df)

,JobTitle,Gender,Salary,rank
0,Accountant,Female,47000,1
1,Accountant,Male,42000,2
2,Data Analyst,Male,95409,1
3,Finance Analyst,Female,93000,1
4,HR,Female,72000,1
5,HR,Male,50000,2
6,IT Support,Female,65000,1
7,Marketing Manager,Male,87000,1
8,Operations Manager,Male,98577,1
9,Receptionist,Female,36000,1


In [138]:
query = """
SELECT dem.EmployeeID, 
    dem.FirstName, 
     dem.LastName, 
       dem.Gender, 
       sal.Salary, 
ROW_NUMBER() OVER(PARTITION BY dem.Gender ORDER BY sal.Salary DESC) as row_num,
RANK() OVER(PARTITION BY dem.Gender ORDER BY sal.Salary DESC) as rank_num
FROM EmployeeDemographics AS dem
JOIN EmployeeSalary AS sal
    ON dem.EmployeeID = sal.EmployeeID;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,FirstName,LastName,Gender,Salary,row_num,rank_num
0,1011,Jane,Johnson,Female,113332,1,1
1,1013,Emily,Brown,Female,100110,2,2
2,1019,Laura,Martinez,Female,93000,3,3
3,1015,Sarah,Garcia,Female,72000,4,4
4,1017,Anna,Davis,Female,65000,5,5
5,1004,Angela,Martin,Female,47000,6,6
6,1007,Meredith,Palmer,Female,41000,7,7
7,1002,Pam,Beasley,Female,36000,8,8
8,1016,James,Miller,Male,110500,1,1
9,1014,David,Jones,Male,98577,2,2


**DENSE_RANK()**

In [142]:
query = """
SELECT dem.EmployeeID, 
    dem.FirstName, 
     dem.LastName, 
       dem.Gender, 
       sal.Salary, 
ROW_NUMBER() OVER(PARTITION BY dem.Gender ORDER BY sal.Salary DESC) as row_num,
RANK() OVER(PARTITION BY dem.Gender ORDER BY sal.Salary DESC) as rank_num,
DENSE_RANK() OVER(PARTITION BY dem.Gender ORDER BY sal.Salary DESC) as dense_rank
FROM EmployeeDemographics AS dem
JOIN EmployeeSalary AS sal
    ON dem.EmployeeID = sal.EmployeeID;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,FirstName,LastName,Gender,Salary,row_num,rank_num,dense_rank
0,1011,Jane,Johnson,Female,113332,1,1,1
1,1013,Emily,Brown,Female,100110,2,2,2
2,1019,Laura,Martinez,Female,93000,3,3,3
3,1015,Sarah,Garcia,Female,72000,4,4,4
4,1017,Anna,Davis,Female,65000,5,5,5
5,1004,Angela,Martin,Female,47000,6,6,6
6,1007,Meredith,Palmer,Female,41000,7,7,7
7,1002,Pam,Beasley,Female,36000,8,8,8
8,1016,James,Miller,Male,110500,1,1,1
9,1014,David,Jones,Male,98577,2,2,2


#### CTEs (Common Table Expressions)

Allow us to define a sub query block that we can reference within the main query.

CTEs are defined by the WITH clause.

CTEs are unique because you only use them immediately after you create it.

In [148]:
query = """
SELECT 
    dem.Gender,
    AVG(sal.Salary) as avg_salary,
    MAX(sal.Salary) as max_salary,
    MIN(sal.Salary) as min_salary,
    COUNT(sal.Salary)
FROM EmployeeDemographics AS dem
JOIN EmployeeSalary AS sal
    ON dem.EmployeeID = sal.EmployeeID
GROUP BY dem.Gender;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,Gender,avg_salary,max_salary,min_salary,COUNT(sal.Salary)
0,Female,70930.250000,113332,36000,8
1,Male,70971.363636,110500,42000,11


With a CTE we are kind of building our own table that can be quered later.

In [162]:
query = """

WITH CTE_Example AS
(
    SELECT 
        dem.Gender,
        AVG(sal.Salary) as avg_salary,
        MAX(sal.Salary) as max_salary,
        MIN(sal.Salary) as min_salary,
        COUNT(sal.Salary) as count
    FROM EmployeeDemographics AS dem
    JOIN EmployeeSalary AS sal
        ON dem.EmployeeID = sal.EmployeeID
    GROUP BY dem.Gender
)

SELECT AVG(avg_salary) as total_avg
FROM CTE_Example;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,total_avg
0,70950.806818


In [170]:
query = """

WITH CTE_Example AS
(
    SELECT 
        dem.Gender,
        AVG(sal.Salary) as avg_salary,
        MAX(sal.Salary) as max_salary,
        MIN(sal.Salary) as min_salary,
        COUNT(sal.Salary) as count
    FROM EmployeeDemographics AS dem
    JOIN EmployeeSalary AS sal
        ON dem.EmployeeID = sal.EmployeeID
    GROUP BY dem.Gender
)

SELECT dem.FirstName, dem.Gender, CTE_Example.avg_salary
FROM EmployeeDemographics as dem
JOIN CTE_Example
    ON dem.Gender = CTE_Example.Gender;
""" 
df = pd.read_sql_query(query, conn)
display(df)

,FirstName,Gender,avg_salary
0,Jim,Male,70971.363636
1,Pam,Female,70930.250000
2,Dwight,Male,70971.363636
3,Angela,Female,70930.250000
4,Toby,Male,70971.363636
5,Michael,Male,70971.363636
6,Meredith,Female,70930.250000
7,Stanley,Male,70971.363636
8,Kevin,Male,70971.363636
9,John,Male,70971.363636


The purpose of CTE's is to facilitate further advanced calculations that are otherwise impossible within a single query.

**Creating multiple CTEs**

We can perform joins on the two table common expressions

In [161]:
query = """

WITH CTE_Example AS
(
    SELECT 
        dem.EmployeeID,
        dem.Gender
    FROM EmployeeDemographics AS dem
    WHERE dem.Age > 30   
),
CTE_Example2 AS 
(
    SELECT EmployeeID, Salary
    FROM EmployeeSalary
    WHERE Salary > 50000
)

SELECT *
FROM CTE_Example
JOIN CTE_Example2
    ON CTE_Example.EmployeeID = CTE_Example2.EmployeeID
""" 
df = pd.read_sql_query(query, conn)
display(df)

,EmployeeID,Gender,EmployeeID,Salary
0,1006,Male,1006,65000
1,1010,Male,1010,95409
2,1012,Male,1012,76199
3,1014,Male,1014,98577
4,1015,Female,1015,72000
5,1016,Male,1016,110500
6,1018,Male,1018,87000
